In [ ]:
LIB_DIR = '~/kcpark/package/tensorflow-glove'
WORK_DIR = '~/kcpark/workspace/dl4rs/embedding'

In [ ]:
import sys
sys.path.append(LIB_DIR)
import numpy as np
from scipy.spatial.distance import cdist
import tf_glove

In [ ]:
# model
model = tf_glove.GloVeModel(
    embedding_size=100,    # 50-300
    context_size=10,
    max_vocab_size=100000,
    min_occurrences=5,
    batch_size=512,
    learning_rate=0.05,
)

In [ ]:
# load data
def get_corpus(corpus_file):
    for l in open(corpus_file):
        gdid, terms = l.strip().split('\t')
        yield eval(terms)    # ['박근혜', '최순실', ...]
corpus = get_corpus(WORK_DIR+'/data/corpus.txt')
model.fit_to_corpus(corpus)

In [ ]:
# train
model.train(
    log_dir=WORK_DIR+'/logs',
    num_epochs=100,
    summary_batch_interval=1000,
)

In [ ]:
# nn (test)
v = model.embedding_for('박근혜')    # model.embedding[model.id_for_word('최순실')]
W = model.embeddings
similarity = cdist(W, v.reshape(1, -1), 'cosine').reshape(-1)
for idx in similarity.argsort()[1:10+1]:
    print model.words[idx], similarity[idx]

In [ ]:
# save result
np.save(WORK_DIR+'/models/e01', model.embeddings)
with open(WORK_DIR+'/models/v01.txt', 'w') as vocab:
    vocab.write('\n'.join(model.words))